# Data Processing

MIT License

Copyright (c) 2023 - J.R.Verbiest

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

---

Dataset: [TRIPOD—A Treadmill Walking Dataset with IMU, Pressure-Distribution and Photoelectric Data for Gait Analysis](https://www.mdpi.com/2306-5729/6/9/95)

In [ ]:
# --------------------------------------------------------------------------------------------
# parameters
# --------------------------------------------------------------------------------------------
subject = "<Sub_..>"
walking_speed = "<PWS-..>"
sensor_location = "RF"

SAVEDATA = 0
# --------------------------------------------------------------------------------------------

---

In [ ]:
%load_ext autoreload
%autoreload 2
# --------------------------------------------------------------------------------------------

import os
import json

import pandas as pd 
import numpy as np

from bokeh.io import show, output_notebook
from bokeh.plotting import figure

import matplotlib.pyplot as plot

import logging
logging.basicConfig(level=logging.INFO, format="%(asctime)-15s %(message)s")
logger = logging.getLogger()

import helper

# --------------------------------------------------------------------------------------------
# read config stuff
# --------------------------------------------------------------------------------------------
import yaml
from yaml.loader import SafeLoader
with open(os.path.join(os.getcwd(), "config.yaml")) as yaml_file:
    config = yaml.load(yaml_file, Loader=SafeLoader)
yaml_file.close()

# --------------------------------------------------------------------------------------------

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

output_notebook()

In [ ]:
fs_imu = config['sensor']['fs_imu']
fs_imu_resample = config['sensor']['fs_imu_resample']

range_acc_g = config['sensor']['range_acc_g']
range_gyr_deg = config['sensor']['range_gyr_deg']

filter_cutoff = config['filter']['filter_cutoff']
filter_order = config['filter']['filter_order']

signal_duration = config['parameters']['signal_duration']
gait_event_detection_threshold = config['gait_event_detection']['gait_event_detection_threshold']

device_type = config['device_type']

---

### Load data

In [ ]:
imu_file =os.path.join(config["path"]["raw_data"], subject, walking_speed, 'IMU', sensor_location + ".csv")
logger.info("imu file: %s" %imu_file)

df_imu = (pd.read_csv(imu_file, skiprows=7, index_col=False, names=['Time', 'Gyro X', 'Gyro Y', 'Gyro Z', 'Accel X', 'Accel Y', 'Accel Z', 'Pressure', 'Temperature', 'Quat W', 'Quat X', 'Quat Y', 'Quat Z'])
          .pipe(helper.select_column, column = ['Time', 'Gyro X', 'Gyro Y', 'Gyro Z', 'Accel X', 'Accel Y', 'Accel Z'])
          .rename(columns = {
              'Accel X': 'Ax',
              'Accel Y': 'Ay',
              'Accel Z': 'Az',
              'Gyro X':  'Gx',
              'Gyro Y':  'Gy',
              'Gyro Z':  'Gz'})
              )

df_imu.head()

### A. Normalization

In [ ]:
df_imu_norm = (df_imu
               .pipe(helper.normalize_imu_data, range_acc_g, range_gyr_deg))

df_imu_norm.head()

### B. Downsampling

In [ ]:
df_imu_resample = (df_imu_norm
                   .pipe(helper.resample_imu_data, fs_imu, fs_imu_resample))

graph = figure(
        title='IMU norm (red) - IMU resample (blue)',
        sizing_mode="stretch_width",
        height=500,
        x_axis_label='time [s]',
        y_axis_label='Gx [rad/s]',
        toolbar_location = "below"
    )
        
graph.line(df_imu_norm['Time'], -df_imu_norm["Gx"], line_color='red', line_width=2)
graph.circle(df_imu_norm['Time'], -df_imu_norm["Gx"], size = 3,  line_color = 'red', fill_color='red')

graph.line(df_imu_resample['Time'], -df_imu_resample["Gx"], line_color='blue', line_width=2)
graph.circle(df_imu_resample['Time'], -df_imu_resample["Gx"], size = 3,  line_color = 'blue', fill_color='blue')
show(graph)

df_imu_resample.head()

### C - Filtering

In [ ]:
df_imu_lpf = (df_imu_resample
              .pipe(helper.lpf_signal, fs=fs_imu_resample, cutoff=filter_cutoff, order = filter_order)
             )

graph = figure(
        title='IMU resample (blue) - IMU filtered (green)',
        sizing_mode="stretch_width",
        height=500,
        x_axis_label='time [s]',
        y_axis_label='Gx [rad/s]',
        toolbar_location = "below"
    )
        
graph.line(df_imu_resample['Time'], -df_imu_resample["Gx"], line_color='blue', line_width=2)
graph.circle(df_imu_resample['Time'], -df_imu_resample["Gx"], size = 3,  line_color = 'blue', fill_color='blue')

graph.line(df_imu_lpf['Time'], -df_imu_lpf["Gx"], line_color='green', line_width=2)
graph.circle(df_imu_lpf['Time'], -df_imu_lpf["Gx"], size = 3,  line_color = 'green', fill_color='green')

show(graph)

### D - Gait Cycle Segmentation

#### get first IC

In [ ]:
df_first_ic = pd.read_csv(os.path.join(config["path"]["raw_data"], "SyncInfo.csv"))
first_ic = helper.get_first_ic(df_first_ic, subject, walking_speed)

logger.info('First IC (R-foot) %s' %first_ic)

#### ground truth (OptoGait)

In [ ]:
path_optogait = os.path.join(config["path"]["raw_data"], subject, walking_speed, 'OptoGait/optogait.csv')
logger.info('Path OptoGait %s' %path_optogait)

df_optogait_data = pd.read_csv(path_optogait, index_col=False)

metadata_seg = helper.get_optogait_data(df_optogait_data, first_ic, signal_duration, sensor_location)
metadata_seg.head()

In [ ]:
metadata_seg.tail()

In [ ]:
df_optogait_data[['L/R', 'Split', 'Stride', 'StrideTime\\Cycle']].head(10)

#### Gait Event Detection

In [ ]:
IC = helper.gait_event_detection(signal = -df_imu_lpf['Gx'], threshold = gait_event_detection_threshold)
IC_sec = (IC/fs_imu_resample).tolist()
IC_sec.insert(0, first_ic)
IC_sec = np.asarray(IC_sec)

delta_stride_time = (config['parameters']['perc_stride_time']/100)*metadata_seg['stride time (OptoGait) [sec]'].describe()['mean']
logger.info('delta_stride_time %s' %delta_stride_time)
IC = helper.drop_false_IC(delta_stride_time, IC_sec)

In [ ]:
if len(IC) > metadata_seg.shape[0]: IC_len = metadata_seg.shape[0]
else: IC_len = len(IC)

logger.info('len(IC) %s' %len(IC))
logger.info('metadata_seg.shape[0] %s' %metadata_seg.shape[0])

metadata_seg = metadata_seg[:IC_len]
metadata_seg['IC_n (GaitED) [sec]'] = IC[:IC_len]
metadata_seg['IC_n-1 (GaitED) [sec]'] = metadata_seg['IC_n (GaitED) [sec]'].shift(1)
metadata_seg['IC_n-1 (GaitED) [sec]'].iloc[0] = first_ic
metadata_seg['stride time (GaitED) [sec]'] = metadata_seg['IC_n (GaitED) [sec]']-metadata_seg['IC_n-1 (GaitED) [sec]']
metadata_seg['(stride time (GaitED) - stride time (OptoGait)) [ms]'] = 1000*round(metadata_seg['stride time (GaitED) [sec]']-metadata_seg['stride time (OptoGait) [sec]'],3)
metadata_seg = metadata_seg.reindex(columns=['stride length (OptoGait) [cm]', 'stride time (OptoGait) [sec]', 'IC_n-1 [sec]', 'IC_n [sec]', 'stride time (GaitED) [sec]', 'IC_n-1 (GaitED) [sec]', 'IC_n (GaitED) [sec]', '(stride time (GaitED) - stride time (OptoGait)) [ms]'])

metadata_seg

In [ ]:
metadata_seg['(stride time (GaitED) - stride time (OptoGait)) [ms]'].describe()

In [ ]:
metadata_seg['stride time (GaitED) [sec]'].describe()

In [ ]:
helper.plot_imu_axis(imu_norm = df_imu_norm, imu_lpf = df_imu_lpf, firstIC = first_ic, IC_ref = metadata_seg['IC_n [sec]'], IC_gs = metadata_seg['IC_n (GaitED) [sec]'], axis = 'Gx [rad/s]')
helper.plot_imu_axis(imu_norm = df_imu_norm, imu_lpf = df_imu_lpf, firstIC = first_ic, IC_ref = metadata_seg['IC_n [sec]'], IC_gs = metadata_seg['IC_n (GaitED) [sec]'], axis = 'Gy [rad/s]')
helper.plot_imu_axis(imu_norm = df_imu_norm, imu_lpf = df_imu_lpf, firstIC = first_ic, IC_ref = metadata_seg['IC_n [sec]'], IC_gs = metadata_seg['IC_n (GaitED) [sec]'], axis = 'Gz [rad/s]')
helper.plot_imu_axis(imu_norm = df_imu_norm, imu_lpf = df_imu_lpf, firstIC = first_ic, IC_ref = metadata_seg['IC_n [sec]'], IC_gs = metadata_seg['IC_n (GaitED) [sec]'], axis = 'Ax [g]')
helper.plot_imu_axis(imu_norm = df_imu_norm, imu_lpf = df_imu_lpf, firstIC = first_ic, IC_ref = metadata_seg['IC_n [sec]'], IC_gs = metadata_seg['IC_n (GaitED) [sec]'], axis = 'Ay [g]')
helper.plot_imu_axis(imu_norm = df_imu_norm, imu_lpf = df_imu_lpf, firstIC = first_ic, IC_ref = metadata_seg['IC_n [sec]'], IC_gs = metadata_seg['IC_n (GaitED) [sec]'], axis = 'Az [g]')

#### Gait Segmentation & Labelling

In [ ]:
gait_cycle = []
proc_metadata = []

for cycle in range(metadata_seg.shape[0]): # metadata_seg.shape[0] = number of gait cycles (IC-IC) starting from zero
    #logger.info('Gait cycle %s' %cycle)
    gait_cycle.append(helper.get_gait_cycle(df_imu_lpf, metadata_seg['IC_n-1 (GaitED) [sec]'][cycle], metadata_seg['IC_n (GaitED) [sec]'][cycle]))
    data_axes = gait_cycle[cycle].drop(columns = ['Time'])
    
    # check if we have a complete gait cycle
    nmb_of_samples_cycle = data_axes.shape[0]
    if str(metadata_seg.iloc[cycle]['stride time (GaitED) [sec]']) == 'nan': 
        # if stride time is nan -> nmb_of_samples_cycle_ref = 0 -> cycle_complete = 'FALSE'
        nmb_of_samples_cycle_ref = 0
    else:
        nmb_of_samples_cycle_ref = int(round(metadata_seg.iloc[cycle]['stride time (GaitED) [sec]']*config['sensor']['fs_imu_resample'],0))
        
    if abs(nmb_of_samples_cycle_ref - nmb_of_samples_cycle) > config['parameters']['delta']:
        cycle_complete = 'FALSE'
    else: 
        cycle_complete = 'TRUE'
        
    x_data = helper.json_sensordata(
        device_type=device_type, 
        sensors=[
            { "name": "Gx", "units": "rad/s" },
            { "name": "Gy", "units": "rad/s" },
            { "name": "Gz", "units": "rad/s" },
            { "name": "Ax", "units": "g" },
            { "name": "Ay", "units": "g" },
            { "name": "Az", "units": "g" }
        ], 
        sensors_values=data_axes.values.tolist(), 
        interval_ms=round(1000*(1/config['sensor']['fs_imu_resample']),6)
    )
    
    json_filename = subject+'-'+walking_speed+'-'+sensor_location+'-'+str(cycle)+'-stride-length-cm-'+str(int(metadata_seg.iloc[cycle]['stride length (OptoGait) [cm]']))+'.json'
    
    if SAVEDATA == True:
        with open(os.path.join(config['path']['processed_data'],subject,walking_speed,json_filename), 'w') as json_file:
            json.dump(x_data, json_file)
        json_file.close()
    
    proc_metadata.append({
        'filename':                       str(json_filename),
        'subject':                        subject,
        'PWS':                            walking_speed,
        'gait cycle':                     cycle,
        'fs [Hz]':                        config['sensor']['fs_imu_resample'],
        'cycle complete':                 cycle_complete,
        'stride length (OptoGait) [cm]':  metadata_seg.iloc[cycle]['stride length (OptoGait) [cm]'],
        'stride time (OptoGait) [sec]':   metadata_seg.iloc[cycle]['stride time (OptoGait) [sec]'],
        'IC_n-1 [sec] (OptoGait) [sec]':  metadata_seg.iloc[cycle]['IC_n-1 [sec]'],
        'IC_n [sec] (OptoGait) [sec]':    metadata_seg.iloc[cycle]['IC_n [sec]'],
        'stride time (GaitED) [sec]':     metadata_seg.iloc[cycle]['stride time (GaitED) [sec]'],
        'IC_n-1 (GaitED) [sec]':          metadata_seg.iloc[cycle]['IC_n-1 (GaitED) [sec]'],        
        'IC_n (GaitED) [sec]':            metadata_seg.iloc[cycle]['IC_n (GaitED) [sec]']      
    })
    
proc_dataset_meta = pd.DataFrame.from_records(proc_metadata)

json_data = proc_dataset_meta.to_json(orient="records")

if SAVEDATA == True:
    with open(os.path.join(config['path']['proc_dataset'],subject+'-'+walking_speed+'-'+sensor_location+'.json'), 'w') as f:
        f.write(json_data)
    f.close()
    
proc_dataset_meta.head()

---